In [ ]:
"record_no", "parent_id", "rollup_no", "doc_type", "stateno", "name_use_cd", "f_name", "m_name", "l_name", "addr_type_cd", "addr_original_type_cd", "address_dt", "state", "cntry", "cntry_usd", "birth_sex", "birth_dt", "birth_cntry", "birth_usd", "vital_status", "swm", "swf", "idu", "rcvd_clot_factor", "factor_type", "factor_dt", "sexidu", "sexbi", "sexhemo", "sex_transf", "sex_transpl", "sex_aidshiv", "rcvd_transf", "first_trans_dt", "last_trans_dt", "rcvd_transpl", "hlth_worker", "occupation", "other_risk", "nir", "nir_dt", "refer_obgyn"

In [161]:
dfQA.to_csv(f'eHARSLabQAOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)

In [10]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as dt
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date
import settings
import errhandler as eh
import eHSQL
import re
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

con = None
try:
    global conn, recordlist
    conn = False
    logging.basicConfig(filename = 'eHARSExtract.log',
                       filemode='a',
                       format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                       datefmt='%H:%M:%S',
                       level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')

    #Opens connection to the Oracle DB via the stored config file in the config folder
    def open_connection():
        global driver, database, uid, pwd, connection, con
        #connection = 'DRIVER={Oracle in instantclient_21_6};DBQ=NDEDSS_DEVsvc.itd.nd.gov;UID=NDDH_EDSS_Intr;PWD=QRSTUVW1'
        connection = f'{settings.username}/{settings.password}@{settings.servicename}'
        con = O.connect(connection)
        #con = pyodbc.connect(connection)
        return con

    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT * FROM v$version")
        row = cur.fetchone()
        print("Connection established to: ", row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return

    def close_connection():
        con.close()
        return

    def tablecount(con):
        cur = con.cursor()
        cur.execute(eHSQL.SQLCaseCount)
        output = cur.fetchone()
        #print(output)
        if output[0] == 0:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return False
        else:
            print(f'\n{output[0]} Existing Records\n', end = "\r")
            logging.info(f'\n{output[0]} Existing records\n')
            cur.close()
            con.commit()
            return True

    def tablecontent(con):
        global recordlist
        cur = con.cursor()
        SQL = eHSQL.SQLHIVData
        cur.execute(SQL)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df = pd.read_sql(SQL, con)
            display(df)
        output = cur.fetchall()
        output = output[0:]
        output = str(tuple(list(list(zip(*output))[0])))
        #print(output)
        #print(f'\nRecord List: {output}\n', end = "\r")
        logging.info(f'\nRecord List: {output}\n')
        recordlist = output
        cur.close()
        con.commit()
        return

    def get_data(con, recordlist):
        cur = con.cursor()
        print(recordlist)
        SQL = f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist
        cur.execute(SQL)
        global df
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
            warnings.simplefilter('ignore', FutureWarning)
            
            df = pd.read_sql(SQL, con)
            #display(df)
            
            df_test = df.pivot(index = ['INVRESULTUNID'], columns = 'NAME', values=['VALUE'])
            #display(df_test)
            df_test.columns = df_test.columns.droplevel()
            df_test = df_test.rename_axis(None, axis = 1)
            df = df.drop(columns = ['NAME', 'VALUE']).drop_duplicates()
            
            df = pd.merge(df, df_test, how ='outer', on = 'INVRESULTUNID')
            
            #display(df_test)

            df['record_no'] = np.arange(len(df))+1
            rcount = df.pop('record_no')
            df.insert(0, rcount.name, rcount)
            df['INVUNID'] = df['INVUNID'].astype('str')
            df['binary'] = df['INVUNID'].shift(1, fill_value = df['INVUNID'].head(1)) != df['INVUNID']
            df['parent_id'] = df['INVUNID'].groupby((df['binary']).cumsum()).cumcount()+1
            df['U_ID'] = df.groupby('INVUNID', sort=False).ngroup()+1
            df['rollup_no'] = np.nan
            df.loc[df['parent_id'] == 1, 'rollup_no'] = df['record_no']
            df['rollup_no'] = df['rollup_no'].ffill()
            df['rollup_no'] = df['rollup_no'].astype('int')
            
            df.set_index('record_no', inplace = True)

            df = df.drop(columns = ['binary'])
            
            col = df.pop('U_ID')
            df.insert(1, col.name, col)
            
            col = df.pop('parent_id')
            df.insert(2, col.name, col)
            
            col = df.pop('rollup_no')
            df.insert(3, col.name, col)
            
            recordadded = False
            isGenoType = False
            genotypeid = ['61199-6','34700-5','45176-5','45175-7','33630-5','53798-5','90903-6','81122-4']
            
            df['doc_type'] = "004"
            df['source'] = "A05.03"
            df['surv_method'] = "P"
            
            datelist = ('CREATE_DATE', 'MODIFICATION_DATE', 'BIRTH_DATE')
            date_format = "%m/%d/%Y"
            
            for date in enumerate(datelist):
                if date[1] in df:
                    df[date[1]] = df[date[1]].dt.strftime(date_format)
                else:
                    continue
                    
            df['enter_dt'] = df['CREATE_DATE']
            df['rcvd_dt'] = df['CREATE_DATE']
            df['m_name'] = ''
            df.loc[df['parent_id'] == 1, 'f_name'] = df['FIRST_NAME']
            df.loc[df['parent_id'] == 1, 'l_name'] = df['LAST_NAME']
            df.loc[df['parent_id'] == 1, 'birth_dt'] = df['BIRTH_DATE']
            df.loc[df['parent_id'] == 1, 'name_use_cd'] = 'L'
            df['isGenoType'] = False
            df['facility_usd'] = ''
            df['facility_type'] = ''
            df['specimen_id'] = ''
            df['lab_specific_note'] = ''
            df['comments'] = ''
            df['notes'] = ''
            df['name_use_cd'] = ''
            df['addr_type_cd'] = ''
            df['cntry_usd'] = ''
            df['birth_usd'] = ''
            df['factor_type'] = ''
            df['sexbi'] = ''
            df['nir_dt'] = ''
            df['facility_name'] = ''
            df['facility_type'] = ''
            df['facility_city'] = ''
            df['facility_st'] = ''
            df['facility_ctry'] = ''
            df['facility_usd'] = ''
            df['facility_phone'] = ''
            df['refer_obgyn'] = ''
            
            try:
                df['notes'] = df['Notes']
            except KeyError:
                df['notes'] = ''
                pass
            try:
                df['lab_clia_no'] = df['CLIA']
            except KeyError:
                df['lab_clia_no'] = ''
                pass
            try:
                df['accession_no'] = df['SpecimenNumber']
            except KeyError:
                df['accession_no'] = ''
                pass
            try:
                df['colldate'] = df['SpecimenDate']
            except KeyError:
                df['colldate'] = ''
                pass
                
            df['result_dt'] = ''
            df['specimen_recieve_dt'] = ''
            df['specimentype'] = ''
            df['lab_result'] = ''
            df['lab_units'] = ''
            
            df['lab_test_name'] = df['Test']
            df['genotype_sequence'] = ''
            df['starhs_sample_id'] = ''
            df['sreason'] = ''
            
            #display(df)
                
            #Interpretation condition values
            interpret1 = ['20447-9','25836-8','29541-0','24013-5','25835-0','59419-2','70241-5']
            statuslist = ['24013-5','29541-0','25835-0']
            interpret2 = ['29541-0','24013-5']

            PCT = ['8128-1','8123-2','32516-7']
            CNT = ['8127-3','24467-3','32515-9']
            CML = ['20447-9','25836-8','24013-5','59419-2','70241-5','25835-0']
            LOG = ['29541-0']
            
            units = ['PCT','CNT','Copies/ML','Log']
            
            POS = ['10828004','G-A200','11214006','G-A497','261420005','260373001']
            NEG = ['260415000','260385009']
            IND = ['82334004']
            
            result = ['POS', 'NEG', 'IND']
            
            #Identify HIV/STD cases
            for gid in enumerate(genotypeid):
                df.loc[df['Test'] == str(gid[1]), 'isGenoType'] = True
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(PCT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[0])
            
            for vals in enumerate(CNT):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[1])
              
            for vals in enumerate(CML):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[2])
            
            for vals in enumerate(LOG):
                df.loc[df['Test'] == str(vals[1]), 'lab_units'] = str(units[3])
                
            #display(df)
                
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'lab_specific_note'] = df['Comments'][0:100]
            df.loc[(df['Comments'] != 'None') | (df['notes'] != ''),'comments'] = df['Comments'][0:255]
            
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True),'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['ResultValue'].str.contains('ND00')) & (df['isGenoType'] == True),'lab_units'] = 'BP'
            
            df.loc[(df['isGenoType'] == True), 'genotype_sequence'] = df['EXTENDED_VALUE']
            df.loc[(df['isGenoType'] == True),'lab_units'] = 'BP'
            
            #Identify UOM (Unit of Measurement)
            for vals in enumerate(POS):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[0])
            for vals in enumerate(NEG):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[1])
            for vals in enumerate(IND):
                df.loc[df['Result'] == str(vals[1]), 'lab_result'] = str(result[2])
                    
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains(';').items():
                    df['ResultValue'] = df['ResultValue'].str.replace('^[^;]*;','').str.replace('[A-Z]','')
            
            if df['ResultValue'].iteritems() != 'None':
                if df['ResultValue'].str.contains('<').items():
                    df['ResultValueTemp'] = df['ResultValue'].str.replace("<","")
                    
            df.loc[(df['lab_result'] == 'None') & (df['isGenoType'] == False), 'lab_result'] = df['CASE_ID']
            df.loc[(df['lab_result'] == '') ,'lab_result'] = df['ResultValueTemp']

            for vals in enumerate(interpret1):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')), 'interpretation'] = '<'
                
                df['result_range_upper'] = 10000000
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValue'].str.contains('<')) & ('.' not in df['lab_result']), 'result_range_lower'] = df['lab_result']
                
            for vals in enumerate(statuslist):
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('DETECTED')),'lab_result'] = '0'
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT DETECTED')),'lab_result'] = None
                df.loc[(df['Test'] == str(vals[1])) & (df['ReferenceRange'].str.contains('NOT QUANTIFIED')),'lab_result'] = None
            
            for vals in enumerate(interpret2):
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None'),'interpretation'] = '<'
                df.loc[(df['Test'] == str(vals[1])) & (df['ResultValueTemp'].iteritems() != 'None') & (df['ResultValueTemp'] >= '0'),'interpretation'] = '='
            
            df['result_range_lower'] = 20
            df.loc[df['Test'] == '59419-2', 'result_range_lower'] = 40
            df['result_range_upper'] = 10000000
            
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] < '20'), 'interpretation'] = '<'
            df.loc[((df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] >= '20') & (df['ResultValueTemp'] <= '10000000')), 'interpretation'] = '='
            df.loc[(df['ResultValueTemp'] != 'None') & (df['ResultValueTemp'] > '10000000'), 'interpretation'] = '>'
            
            df.loc[df['result_range_lower'] == '1.6', 'result_range_lower'] = '' 
            df.loc[df['lab_test_name'] == '20447-9', 'lab_test_name'] = '59419'            
            df.loc[((df['lab_test_name'] == '20447-9') | (df['lab_test_name'] == '25836-8') | (df['lab_test_name'] == '29541-0') | (df['lab_test_name'] == '70241-5') | (df['lab_test_name'] == '25835-0')) , 'lab_test_name'] = '74854'        
            df.loc[((df['lab_test_name'] == '24467-3') | (df['lab_test_name'] == '32515-9')) , 'lab_test_name'] = '24467'  
            df.loc[((df['lab_test_name'] == '8123-2') | (df['lab_test_name'] == '32516-7')) , 'lab_test_name'] = '32516-7'
            df['lab_result'] = df['lab_result'].str.replace('=','')
            
            #display(df)
                   
            #df = df.drop(columns = ['ResultValueTemp',
            #                        'INVUNID',
            #                        'INVRESULTUNID',
            #                        'CREATE_DATE', 
            #                        'MODIFICATION_DATE', 
            #                        'ReferenceRange', 
            #                        'Result', 
            #                        'ResultStatus', 
            #                        'ResultUnits', 
            #                        'TestLocalCode', 
            #                        'TestLocalDescription', 
            #                        'TestValue'])
            #
            #df = df[['U_ID',
            #         'CASE_ID',
            #         'parent_id',
            #         'rollup_no',
            #         'doc_type',
            #         'STATENO',
            #         'rcvd_dt',
            #         'source',
            #         'surv_method',
            #         'name_use_cd',
            #         'f_name',
            #         'm_name',
            #         'l_name',
            #         'birth_dt',
            #         'facility_name',
            #         'facility_type',
            #         'facility_city',
            #         'facility_st',
            #         'facility_ctry',
            #         'facility_usd',
            #         'facility_phone',
            #         'lab_clia_no',
            #         'specimen_id',
            #         'accession_no',
            #         'colldate',
            #         'result_dt',
            #         'specimen_recieve_dt',
            #         'lab_test_name',
            #         'specimentype',
            #         'lab_result',
            #         'lab_units',
            #         'interpretation',
            #         'result_range_lower',
            #         'result_range_upper',
            #         'genotype_sequence',
            #         'starhs_sample_id',
            #         'sreason',
            #         'lab_specific_note',
            #         'comments']]
        
            #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            #    display(df)
            
        #print(f'{eHSQL.SQLLabFields}' + 'and cs.case_id in ' + recordlist)
        output = cur.fetchall()
        #print(output)
        #print(f'\n{output}\n', f'\n{type(output)}\n' ,end = "\r")
        logging.info(f'\nData Succesfully Prepped.\n')
        cur.close()
        con.commit()
        return df
    
    def get_QandA(con, recordlist):
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQandA}' + recordlist
        cur.execute(SQL)
        global dfQA
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            dfQA = pd.read_sql(SQL, con)
            
            QSIDList = ['ADMINISTR','DEMOGRAPHIC','RISK','CLINICAL']
            
            dfQA = dfQA.loc[dfQA['QUESTIONSET_ID'].str.contains('|'.join(QSIDList))]
            df_test = dfQA.pivot(index = ['UNID','EXTERNAL_ID','QUESTION_ID'], columns = ['QUESTIONSET_ID'], values=['VALUE'])
            df_test.columns = df_test.columns.droplevel(0)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            df_test = df_test.drop(columns = ['UNID'])
            
            df_test = dfQA.sort_values(by=['QUESTION_ID'])
        
            dfQA['QUESTION_ID'] = dfQA['QUESTION_ID'].str.lower()
            
            display(dfQA)
            
            fieldlist = ['addr_type_cd', 
                         'addr_original_type_cd', 'address_dt', 'country', 'county','assigned_sex', 'birth_date', 'country_of_birth',
                         'hiv_current_status', 'hiv_sex_male', 'hiv_sex_female',
                         'hiv_hemphilia', 'factor_type', 'clotting_factor_date_received', 'sexbi','hetero_contact_transfusion_with_hiv', 
                         'hetero_contact_transplant_with_hiv', 'hetero_contact_with_hiv_not_specified', 
                         'hiv_blood_other','hiv_facility_city','hiv_blood_other_first_date', 'hiv_date_of_death', 'hiv_state_of_death',
                         'hiv_blood_other_last_date','hiv_transplant', 'hlth_worker','hiv_healthcare_setting_specify', 'hiv_healthcare_setting', 'other_document_risk', 'nir', 
                         'nir_dt', 'refer_obgyn','hiv_sex_male_yes_12','hiv_sex_injection_drugs_12mo_yes','hetero_contact_hemophilia_with_hiv']

            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting_specify'), 'QUESTION_ID'] = 'occupation'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting'), 'QUESTION_ID'] = 'hlth_worker'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_first_date'), 'QUESTION_ID'] = 'first_trans_date'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_last_date'), 'QUESTION_ID'] = 'last_trans_date'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('country_of_birth'), 'QUESTION_ID'] = 'birth_cntry'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_yes'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_unknown'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
            dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_yes', 'QUESTION_ID'] = 'sex_idu'
            dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_unknown', 'QUESTION_ID'] = 'sex_idu'
            
            dfQA.loc[dfQA['QUESTION_ID'].str.contains('|'.join(fieldlist))]
            for vals in enumerate(fieldlist):
                dfQA.loc[dfQA['QUESTION_ID'].str.contains(vals[1]), 'QUESTION_ID'] = vals[1]
            
            dfQA.rename(columns={"country":"cntry", "county":"cnty", "assigned_sex":"birth_sex","hiv_current_status":"vital_status",
                                 "hiv_sex_male":"swm", "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu",
                                 "hiv_hemphilia":"rcvd_clot_factor", "clotting_factor_date_received":"factor_dt", "sex_idu":"sexidu",
                                 "hetero_contact_hemophilia_with_hiv":"sexhemo","hetero_contact_transfusion_with_hiv":"sex_transf",
                                 "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                                 "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                                 "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk",
                                 "hiv_facility_city":"facility_type"}, inplace = True)
            
            #display(df_test)

            df_test = dfQA.pivot_table(index = ['EXTERNAL_ID'], columns = ['QUESTION_ID'], values=['VALUE'], aggfunc='sum')
            #display(df_test)
            df_test.columns = df_test.columns.droplevel(0)
            df_test = df_test.rename_axis(None, axis = 1).reset_index()
            dfQA = dfQA.drop(columns = ['UNID','QUESTIONSET_ID','QUESTION_ID','VALUE']).drop_duplicates()

            #display(df_test)
            dfQA = pd.merge(dfQA, df_test, how ='outer', on = 'EXTERNAL_ID')
            
            #display(dfQA)
            
            dfQA['record_no'] = np.arange(len(dfQA))+1
            rcount = dfQA.pop('record_no')
            dfQA.insert(0, rcount.name, rcount)
            dfQA['EXTERNAL_ID'] = dfQA['EXTERNAL_ID'].astype('str')
            dfQA['binary'] = dfQA['EXTERNAL_ID'].shift(1, fill_value = dfQA['EXTERNAL_ID'].head(1)) != dfQA['EXTERNAL_ID']
            dfQA['parent_id'] = dfQA['EXTERNAL_ID'].groupby((dfQA['binary']).cumsum()).cumcount()+1
            dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
            
            dfQA.rename(columns={"CASE_ID": "QUESTIONAIRE_CASE_ID", "EXTERNAL_ID": "CASE_ID"}, inplace = True)
            
            dfQA.set_index('record_no', inplace = True)
            dfQA = dfQA.drop(columns = ['binary'])
            
            col = dfQA.pop('parent_id')
            dfQA.insert(1, col.name, col)
            
            dfQA['parent_id'] = dfQA['parent_id'].astype(str)
            dfQA['CASE_ID'] = dfQA['CASE_ID'].astype(str)
            df['parent_id'] = df['parent_id'].astype(str)
            df['CASE_ID'] = df['CASE_ID'].astype(str)
            df1 = df[['CASE_ID','U_ID','INVUNID','parent_id','rollup_no','doc_type','STATENO','rcvd_dt','source','name_use_cd','f_name','m_name','l_name','birth_dt']]
            df1['parent_id'] = df1['parent_id'].astype(str)
            df1['CASE_ID'] = df1['CASE_ID'].astype(str)
            df1.columns = df1.columns.str.lower()
            dfQA.columns = dfQA.columns.str.lower()
            #display(df1)
            
            dfQA = pd.merge(df1, dfQA, how = 'outer', on=['case_id', 'parent_id']).reset_index(drop=True)
            dfQA = dfQA.drop(columns = 'rollup_no_x')
            
            dfQA['country_usd'] = ''
            dfQA['addr_original_type_cd'] = ''
            dfQA['nir_dt'] = ''
            dfQA['nir'] = 'UNKNOWN'
            #???
            dfQA['address_dt'] = ''
            dfQA['sexbi'] = ''
            dfQA['birth_usd'] = ''
            dfQA['cntry_usd'] = ''
            dfQA['addr_type_cd'] = ''
            dfQA['refer_obgyn'] = ''
            dfQA['factor_type'] = ''
            dfQA['doctype'] = '004'
            
            #display(dfQA)
            
            dfQA = dfQA.sort_values(by=['invunid'])
            dfQA['record_no'] = np.arange(len(dfQA))+1
            rcount = dfQA.pop('record_no')
            dfQA.insert(0, rcount.name, rcount)
            
            dfQA.loc[dfQA['invunid'].isna(), 'invunid'] = dfQA['record_no']
            
            dfQA['invunid'] = dfQA['invunid'].astype('str')
            dfQA['binary'] = dfQA['invunid'].shift(1, fill_value = dfQA['invunid'].head(1)) != dfQA['invunid']
            dfQA['parent_id'] = dfQA['invunid'].groupby((dfQA['binary']).cumsum()).cumcount()+1
            dfQA['U_ID'] = dfQA.groupby('invunid', sort=False).ngroup()+1
            dfQA['rollup_no'] = np.nan
            dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
            dfQA['rollup_no'] = dfQA['rollup_no'].ffill()
            dfQA['rollup_no'] = dfQA['rollup_no'].astype('int')
            #
            dfQA = dfQA.drop(columns = ['rollup_no_y','binary'])
            
            #display(dfQA)
            
            col = dfQA.pop('invunid')
            dfQA.insert(0, col.name, col)
            
            col = dfQA.pop('parent_id')
            dfQA.insert(1, col.name, col)
            
            col = dfQA.pop('U_ID')
            dfQA.insert(2, col.name, col)
            
            dfQA = dfQA.rename(columns=str.lower)
            
            col = dfQA.pop('rollup_no')
            dfQA.insert(3, col.name, col)
            
            col = dfQA.pop('doc_type')
            dfQA.insert(4, col.name, col)
            
            
            dfQA = dfQA.rename(columns={"country":"cntry", "state_x":"state", "zip":"zip_code", "county":"cnty", 
                                        "assigned_sex":"birth_sex", "country_of_birth":"birth_cntry","hiv_current_status":"vital_status",
                                        "hiv_sex_male":"swm", "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu","hiv_hemphilia":"rcvd_clot_factor",
                                        "clotting_factor_date_received":"factor_dt",
                                        "sex_idu":"sexidu", "hetero_contact_transfusion_with_hiv":"sex_transf",
                                        "hetero_contact_hemophilia_with_hiv":"sexhemo",
                                        "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                                        "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                                        "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk",
                                        })
            
            #display(dfQA)
            
            dfQA = dfQA[['record_no','invunid','parent_id','rollup_no','doc_type','stateno','name_use_cd','f_name','m_name',
                         'l_name','addr_type_cd', 'addr_original_type_cd', 'address_dt', 'state', 'cntry', 'cntry_usd',
                         'birth_sex', 'birth_dt','birth_cntry', 'birth_usd', 'vital_status', 'swm',
                         'swf', 'idu', 'rcvd_clot_factor', 'factor_type', 'factor_dt', 'sexidu', 'sexbi', 'sex_transf',
                         'sexhemo','sex_transpl', 'sex_aidshiv', 'rcvd_transf',  'rcvd_transpl', 'hlth_worker', 'occupation',
                         'other_risk', 'nir', 'nir_dt', 'refer_obgyn']]
            
            #'first_trans_dt','last_trans_dt','sexhemo'
            
            #dfQA['parent_id'] = dfQA['parent_id'].astype(str)
            #dfQA['case_id'] = dfQA['case_id'].astype(str)
            #df1 = df[['CASE_ID','U_ID','parent_id','rollup_no','doc_type','STATENO','rcvd_dt','source','name_use_cd','f_name','m_name','l_name','birth_dt']]
            #df1['parent_id'] = df1['parent_id'].astype(str)
            #df1['CASE_ID'] = df1['CASE_ID'].astype(str)
            #df1.columns = df1.columns.str.lower()
        

                        
            #display(dfQA)
            
            dfQA.set_index('record_no', inplace = True)
            
            #dfQA = pd.merge(df1, dfQA, how = 'inner', on=['case_id', 'parent_id']).reset_index(drop=True)
        
            #dfQA = pd.concat([df1, dfQA], keys=['case_id','parent_id'], axis=1)
            
            display(dfQA)
            
    
    def Update_Last_Extract():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLDateUpdate}' + recordlist
        cur.execute(SQL)
        global df2
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df2 = pd.read_sql(SQL, con)
            
        display(df2)
        
        #for ids in enumerate(df2['UNID']):
        #    if ((ids[1] != 'None') | (ids[1] != 0)):
        #        dtu = "Update NDDH_EDSS.IDS_Answer set Value = '" + date.today().strftime('%dd/%mm/%Y') + "', Date_Value = CURRENT_TIMESTAMP where UNID = '" + ids[1] + "'"
        #        cur.execute(dtu)
        #        cur.close()
        #        con.commit()
        
        cur.close()
        con.commit()
        return df2
    
    def GetQuestionSetUNID():
        cur = con.cursor()
        SQL = f'{eHSQL.SQLQuestionID}' + recordlist
        cur.execute(SQL)
        global df3
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            df3 = pd.read_sql(SQL, con)
            
        display(df3)
        
        cur.close()
        con.commit()
        return df3

    open_connection()
    conn = True
    connection_test()
    close_connection()
    conn = False
    
    open_connection()
    conn = True
    records = False
    if tablecount(con) == False:
        print("No New Records.\n", end = "\r")
    else:
        print("\nRecords ready for processing.\n", end = "\r")
        tablecontent(con)
        
        get_data(con, recordlist)
        get_QandA(con, recordlist)
        Update_Last_Extract()
        GetQuestionSetUNID()
        GetGlobalUNID()
        #df.to_csv(f'eHARSLabQAOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)
        #for cases in caselist
        #get_data()en

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
    pass

except (Exception, O.Error) as dberr:
    print(dberr)
    logging.exception("message")
    pass

#except Exception as err:
     #for receivers in eh.settings.ReceiverList:
            #SendEmailMessage(receivers, str(e))

finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()

Connection established to:  Oracle Database 12c EE Extreme Perf Release 12.2.0.1.0 - 64bit Production

6 Existing Records

Records ready for processing.


,CASE_ID,UNID
0,100000571,181062204
1,100000008,2921245
2,100000296,90866775
3,100000381,122928347
4,100000518,166582986
5,100006970,574151293


('100000571', '100000008', '100000296', '100000381', '100000518', '100006970')


,UNID,EXTERNAL_ID,CASE_ID,QUESTIONSET_ID,QUESTION_ID,VALUE
1,2921270,100000008,2921245,ADMINISTRATIVE,all_hiv,YES
2,2921271,100000008,2921245,ADMINISTRATIVE,all_std,YES
3,2921301,100000008,2921245,ADMINISTRATIVE,assigned_to,srenton
4,207961765,100000008,2921245,ADMINISTRATIVE,city_hidden,Bismarck
5,2921285,100000008,2921245,ADMINISTRATIVE,clinical_event_type_lab_test_date,LAB_TEST_DATE
6,207956930,100000008,2921245,ADMINISTRATIVE,date_first_submitted,01/01/1900
7,207956931,100000008,2921245,ADMINISTRATIVE,date_notification_sent,01/01/1900
8,2921275,100000008,2921245,ADMINISTRATIVE,date_transmitted_ehars,01/01/1900
9,2921266,100000008,2921245,ADMINISTRATIVE,disease_hiv,HIV
10,2921273,100000008,2921245,ADMINISTRATIVE,disease_status_unknown,UNKNOWN


C:\Users\gmoye\AppData\Local\Temp\2\ipykernel_2096\1685354206.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['parent_id'] = df1['parent_id'].astype(str)
C:\Users\gmoye\AppData\Local\Temp\2\ipykernel_2096\1685354206.py:483: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['CASE_ID'] = df1['CASE_ID'].astype(str)


,invunid,parent_id,rollup_no,doc_type,stateno,name_use_cd,f_name,m_name,l_name,addr_type_cd,addr_original_type_cd,address_dt,state,cntry,cntry_usd,birth_sex,birth_dt,birth_cntry,birth_usd,vital_status,swm,swf,idu,rcvd_clot_factor,factor_type,factor_dt,sexidu,sexbi,sex_transf,sexhemo,sex_transpl,sex_aidshiv,rcvd_transf,rcvd_transpl,hlth_worker,occupation,other_risk,nir,nir_dt,refer_obgyn
record_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1066930024,1,1,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,6110TEST,UNKNOWN,UNKNOWN,,
2,1066930024,2,1,004,120594,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,,
3,1066979847,1,3,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,6110TEST,UNKNOWN,UNKNOWN,,
4,1066980171,1,4,004,120594,,Contact_test,,Contact_test2,,,,ND,USA,,FEMALE,07/01/2009,USA,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,,NaN,UNKNOWN,,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,6110TEST,UNKNOWN,UNKNOWN,,
5,1066980171,2,4,004,120594,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,,
6,1067036744,1,6,004,201901,,Fred,,Flintstone,,,,ND,USA,,NaN,01/11/2000,NaN,,NaN,YES,NaN,NaN,YES,,01/01/1990,NaN,,YES,NO,NO,YES,NO,YES,NO,TEST,YES,UNKNOWN,,
7,1067036744,2,6,004,201901,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,,
8,1067036789,1,8,004,201901,,Fred,,Flintstone,,,,ND,USA,,NaN,01/11/2000,NaN,,NaN,YES,NaN,NaN,YES,,01/01/1990,NaN,,YES,NO,NO,YES,NO,YES,NO,TEST,YES,UNKNOWN,,
9,1067036789,2,8,004,201901,,NaN,,NaN,,,,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNKNOWN,,


module 'eHSQL' has no attribute 'SQLDateUpdate' line:  641 

 Traceback (most recent call last):
  File "C:\Users\gmoye\AppData\Local\Temp\2\ipykernel_2096\1685354206.py", line 641, in <cell line: 29>
    Update_Last_Extract()
  File "C:\Users\gmoye\AppData\Local\Temp\2\ipykernel_2096\1685354206.py", line 589, in Update_Last_Extract
    SQL = f'{eHSQL.SQLDateUpdate}' + recordlist
AttributeError: module 'eHSQL' has no attribute 'SQLDateUpdate'



In [9]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as dt
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date
import settings
import errhandler as eh
import eHSQL
import re
import os

def get_QandA(con, recordlist):
    cur = con.cursor()
    SQL = f'{eHSQL.SQLQandA}' + recordlist
    cur.execute(SQL)
    global dfQA
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        dfQA = pd.read_sql(SQL, con)
        
        QSIDList = ['ADMINISTR','DEMOGRAPHIC','RISK','CLINICAL']
        
        dfQA = dfQA.loc[dfQA['QUESTIONSET_ID'].str.contains('|'.join(QSIDList))]
        df_test = dfQA.pivot(index = ['UNID','EXTERNAL_ID','QUESTION_ID'], columns = ['QUESTIONSET_ID'], values=['VALUE'])
        df_test.columns = df_test.columns.droplevel(0)
        df_test = df_test.rename_axis(None, axis = 1).reset_index()
        df_test = df_test.drop(columns = ['UNID'])
        
        df_test = dfQA.sort_values(by=['QUESTION_ID'])
    
        dfQA['QUESTION_ID'] = dfQA['QUESTION_ID'].str.lower()
        
        display(dfQA)
        
        fieldlist = ['addr_type_cd', 
                     'addr_original_type_cd', 'address_dt', 'country', 'county','assigned_sex', 'birth_date', 'country_of_birth',
                     'hiv_current_status', 'hiv_sex_male', 'hiv_sex_female',
                     'hiv_hemphilia', 'factor_type', 'clotting_factor_date_received', 'sexbi','hetero_contact_transfusion_with_hiv', 
                     'hetero_contact_transplant_with_hiv', 'hetero_contact_with_hiv_not_specified', 
                     'hiv_blood_other','hiv_facility_city','hiv_blood_other_first_date', 'hiv_date_of_death', 'hiv_state_of_death',
                     'hiv_blood_other_last_date','hiv_transplant', 'hlth_worker','hiv_healthcare_setting_specify', 'hiv_healthcare_setting', 'other_document_risk', 'nir', 
                     'nir_dt', 'refer_obgyn','hiv_sex_male_yes_12','hiv_sex_injection_drugs_12mo_yes','hetero_contact_hemophilia_with_hiv', 'sex_idu_yes_12_yes']

        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting_specify'), 'QUESTION_ID'] = 'occupation'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting'), 'QUESTION_ID'] = 'hlth_worker'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_first_date'), 'QUESTION_ID'] = 'first_trans_date'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_last_date'), 'QUESTION_ID'] = 'last_trans_date'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('country_of_birth'), 'QUESTION_ID'] = 'birth_cntry'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_yes'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_unknown'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
        dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_yes', 'QUESTION_ID'] = 'sex_idu'
        dfQA.loc[dfQA['QUESTION_ID'] == 'sex_idu_unknown', 'QUESTION_ID'] = 'sex_idu'

        
        dfQA.loc[dfQA['QUESTION_ID'].str.contains('|'.join(fieldlist))]
        for vals in enumerate(fieldlist):
            dfQA.loc[dfQA['QUESTION_ID'].str.contains(vals[1]), 'QUESTION_ID'] = vals[1]
        
        dfQA.rename(columns={"country":"cntry", "county":"cnty", "assigned_sex":"birth_sex","hiv_current_status":"vital_status",
                             "hiv_sex_male":"swm", "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu",
                             "hiv_hemphilia":"rcvd_clot_factor", "clotting_factor_date_received":"factor_dt", "sex_idu":"sexidu",
                             "hetero_contact_hemophilia_with_hiv":"sexhemo","hetero_contact_transfusion_with_hiv":"sex_transf",
                             "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                             "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                             "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk",
                             "hiv_facility_city":"facility_type"}, inplace = True)
        
        #display(df_test)

        df_test = dfQA.pivot_table(index = ['EXTERNAL_ID'], columns = ['QUESTION_ID'], values=['VALUE'], aggfunc='sum')
        #display(df_test)
        df_test.columns = df_test.columns.droplevel(0)
        df_test = df_test.rename_axis(None, axis = 1).reset_index()
        dfQA = dfQA.drop(columns = ['UNID','QUESTIONSET_ID','QUESTION_ID','VALUE']).drop_duplicates()

        #display(df_test)
        dfQA = pd.merge(dfQA, df_test, how ='outer', on = 'EXTERNAL_ID')
        
        display(dfQA)
        
        return
        
open_connection()
conn = True
connection_test()
close_connection()
conn = False

open_connection()
conn = True
records = False
if tablecount(con) == False:
    print("No New Records.\n", end = "\r")
else:
    print("\nRecords ready for processing.\n", end = "\r")
    tablecontent(con)
    
    get_data(con, recordlist)
    get_QandA(con, recordlist)
    Update_Last_Extract()
    GetQuestionSetUNID()
    GetGlobalUNID()
    #df.to_csv(f'eHARSLabQAOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)
    #for cases in caselist
    #get_data()en

Connection established to:  Oracle Database 12c EE Extreme Perf Release 12.2.0.1.0 - 64bit Production

6 Existing Records

Records ready for processing.


,CASE_ID,UNID
0,100000571,181062204
1,100000008,2921245
2,100000296,90866775
3,100000381,122928347
4,100000518,166582986
5,100006970,574151293


('100000571', '100000008', '100000296', '100000381', '100000518', '100006970')


,UNID,EXTERNAL_ID,CASE_ID,QUESTIONSET_ID,QUESTION_ID,VALUE
1,2921270,100000008,2921245,ADMINISTRATIVE,all_hiv,YES
2,2921271,100000008,2921245,ADMINISTRATIVE,all_std,YES
3,2921301,100000008,2921245,ADMINISTRATIVE,assigned_to,srenton
4,207961765,100000008,2921245,ADMINISTRATIVE,city_hidden,Bismarck
5,2921285,100000008,2921245,ADMINISTRATIVE,clinical_event_type_lab_test_date,LAB_TEST_DATE
6,207956930,100000008,2921245,ADMINISTRATIVE,date_first_submitted,01/01/1900
7,207956931,100000008,2921245,ADMINISTRATIVE,date_notification_sent,01/01/1900
8,2921275,100000008,2921245,ADMINISTRATIVE,date_transmitted_ehars,01/01/1900
9,2921266,100000008,2921245,ADMINISTRATIVE,disease_hiv,HIV
10,2921273,100000008,2921245,ADMINISTRATIVE,disease_status_unknown,UNKNOWN


ValueError: Function did not transform

In [ ]:
def get_QandA(con, recordlist):
    cur = con.cursor()
    SQL = f'{eHSQL.SQLQandA}' + recordlist
    cur.execute(SQL)
    global dfQA
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)

    dfQA = pd.read_sql(SQL, con)
    
    QSIDList = ['ADMINISTR','DEMOGRAPHIC','RISK','CLINICAL']
    
    dfQA = dfQA.loc[dfQA['QUESTIONSET_ID'].str.contains('|'.join(QSIDList))]
    df_test = dfQA.pivot(index = ['UNID','EXTERNAL_ID','QUESTION_ID'], columns = ['QUESTIONSET_ID'], values=['VALUE'])
    df_test.columns = df_test.columns.droplevel(0)
    df_test = df_test.rename_axis(None, axis = 1).reset_index()
    df_test = df_test.drop(columns = ['UNID'])
    
    df_test = dfQA.sort_values(by=['QUESTION_ID'])
    
    dfQA['QUESTION_ID'] = dfQA['QUESTION_ID'].str.lower()
    
    #display(dfQA)
    
    fieldlist = ['addr_type_cd', 
                 'addr_original_type_cd', 'address_dt', 'country', 'county','assigned_sex', 'birth_date', 'country_of_birth',
                 'hiv_current_status', 'hiv_sex_male', 'hiv_sex_female',
                 'hiv_hemphilia', 'factor_type', 'clotting_factor_date_received', 'sexbi','hetero_contact_transfusion_with_hiv', 
                 'hetero_contact_transplant_with_hiv', 'hetero_contact_with_hiv_not_specified', 
                 'hiv_blood_other','hiv_facility_city','hiv_blood_other_first_date', 'hiv_date_of_death', 'hiv_state_of_death',
                 'hiv_blood_other_last_date','hiv_transplant', 'hlth_worker','hiv_healthcare_setting_specify', 'hiv_healthcare_setting', 'other_document_risk', 'nir', 
                 'nir_dt', 'refer_obgyn','hiv_sex_male_yes_12','hiv_sex_injection_drugs_12mo_yes','hetero_contact_hemophilia_with_hiv']
    
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting_specify'), 'QUESTION_ID'] = 'occupation'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_healthcare_setting'), 'QUESTION_ID'] = 'hlth_worker'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_first_date'), 'QUESTION_ID'] = 'first_trans_date'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_blood_other_last_date'), 'QUESTION_ID'] = 'last_trans_date'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('country_of_birth'), 'QUESTION_ID'] = 'birth_cntry'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_yes'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs_unknown'), 'QUESTION_ID'] = 'hiv_sex_injection_drugs'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('sex_idu_yes'), 'QUESTION_ID'] = 'sex_idu'
    dfQA.loc[dfQA['QUESTION_ID'].str.contains('sex_idu_unknown'), 'QUESTION_ID'] = 'sex_idu'
    
    df2 = dfQA.loc[dfQA['QUESTION_ID'].str.contains('hiv_sex_injection_drugs')]
    display(df2)
    
    df3 = dfQA.loc[dfQA['QUESTION_ID'].str.contains('sex_idu') & ]
    display(df3)
    
    df4 = dfQA.loc[dfQA['QUESTION_ID'].str.contains('sex_idu')]
    display(df4)
    
open_connection()
conn = True
connection_test()
close_connection()
conn = False

open_connection()
conn = True
records = False
if tablecount(con) == False:
    print("No New Records.\n", end = "\r")
else:
    print("\nRecords ready for processing.\n", end = "\r")
    tablecontent(con)
    
    get_data(con, recordlist)
    get_QandA(con, recordlist)
    #Update_Last_Extract()
    #GetQuestionSetUNID()
    #GetGlobalUNID()
    #df.to_csv(f'eHARSLabQAOut_{str(date.today())}.csv', sep = '\t', encoding = 'utf-8', header = True)
    #for cases in caselist
    #get_data()en

In [154]:
        dfQA['record_no'] = np.arange(len(dfQA))+1
        rcount = dfQA.pop('record_no')
        dfQA.insert(0, rcount.name, rcount)
        dfQA['EXTERNAL_ID'] = dfQA['EXTERNAL_ID'].astype('str')
        dfQA['binary'] = dfQA['EXTERNAL_ID'].shift(1, fill_value = dfQA['EXTERNAL_ID'].head(1)) != dfQA['EXTERNAL_ID']
        dfQA['parent_id'] = dfQA['EXTERNAL_ID'].groupby((dfQA['binary']).cumsum()).cumcount()+1
        dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
        
        dfQA.rename(columns={"CASE_ID": "QUESTIONAIRE_CASE_ID", "EXTERNAL_ID": "CASE_ID"}, inplace = True)
        
        dfQA.set_index('record_no', inplace = True)
        dfQA = dfQA.drop(columns = ['binary'])
        
        col = dfQA.pop('parent_id')
        dfQA.insert(1, col.name, col)
        
        dfQA['parent_id'] = dfQA['parent_id'].astype(str)
        dfQA['CASE_ID'] = dfQA['CASE_ID'].astype(str)
        df['parent_id'] = df['parent_id'].astype(str)
        df['CASE_ID'] = df['CASE_ID'].astype(str)
        df1 = df[['CASE_ID','U_ID','INVUNID','parent_id','rollup_no','doc_type','STATENO','rcvd_dt','source','name_use_cd','f_name','m_name','l_name','birth_dt']]
        df1['parent_id'] = df1['parent_id'].astype(str)
        df1['CASE_ID'] = df1['CASE_ID'].astype(str)
        df1.columns = df1.columns.str.lower()
        dfQA.columns = dfQA.columns.str.lower()
        display(df1)
        
        dfQA = pd.merge(df1, dfQA, how = 'outer', on=['case_id', 'parent_id']).reset_index(drop=True)
        dfQA = dfQA.drop(columns = 'rollup_no_x')
        
        dfQA['country_usd'] = ''
        dfQA['addr_original_type_cd'] = ''
        dfQA['nir_dt'] = ''
        dfQA['nir'] = 'UNKNOWN'
        #???
        dfQA['address_dt'] = ''
        dfQA['sexbi'] = ''
        dfQA['birth_usd'] = ''
        dfQA['cntry_usd'] = ''
        dfQA['addr_type_cd'] = ''
        dfQA['refer_obgyn'] = ''
        dfQA['factor_type'] = ''
        dfQA['doctype'] = '004'
        
        #display(dfQA)
        
        dfQA = dfQA.sort_values(by=['invunid'])
        dfQA['record_no'] = np.arange(len(dfQA))+1
        rcount = dfQA.pop('record_no')
        dfQA.insert(0, rcount.name, rcount)
        
        dfQA.loc[dfQA['invunid'].isna(), 'invunid'] = dfQA['record_no']
        
        dfQA['invunid'] = dfQA['invunid'].astype('str')
        dfQA['binary'] = dfQA['invunid'].shift(1, fill_value = dfQA['invunid'].head(1)) != dfQA['invunid']
        dfQA['parent_id'] = dfQA['invunid'].groupby((dfQA['binary']).cumsum()).cumcount()+1
        dfQA['U_ID'] = dfQA.groupby('invunid', sort=False).ngroup()+1
        dfQA['rollup_no'] = np.nan
        dfQA.loc[dfQA['parent_id'] == 1, 'rollup_no'] = dfQA['record_no']
        dfQA['rollup_no'] = dfQA['rollup_no'].ffill()
        dfQA['rollup_no'] = dfQA['rollup_no'].astype('int')
        #
        dfQA = dfQA.drop(columns = ['rollup_no_y','binary'])
        
        #display(dfQA)
        
        col = dfQA.pop('invunid')
        dfQA.insert(0, col.name, col)
        
        col = dfQA.pop('parent_id')
        dfQA.insert(1, col.name, col)
        
        col = dfQA.pop('U_ID')
        dfQA.insert(2, col.name, col)
        
        dfQA = dfQA.rename(columns=str.lower)
        
        col = dfQA.pop('rollup_no')
        dfQA.insert(3, col.name, col)
        
        col = dfQA.pop('doc_type')
        dfQA.insert(4, col.name, col)
        
        
        dfQA = dfQA.rename(columns={"country":"cntry", "state_x":"state", "zip":"zip_code", "county":"cnty", 
                                    "assigned_sex":"birth_sex", "country_of_birth":"birth_cntry","hiv_current_status":"vital_status",
                                    "hiv_sex_male":"swm", "hiv_sex_female":"swf", "hiv_sex_injection_drugs":"idu","hiv_hemphilia":"rcvd_clot_factor",
                                    "clotting_factor_date_received":"factor_dt",
                                    "sex_idu":"sexidu", "hetero_contact_transfusion_with_hiv":"sex_transf",
                                    "hetero_contact_hemophilia_with_hiv":"sexhemo",
                                    "hetero_contact_transplant_with_hiv":"sex_transpl","hetero_contact_with_hiv_not_specified":"sex_aidshiv",
                                    "hiv_blood_other":"rcvd_transf","hiv_blood_other_first_date":"first_trans_dt","hiv_blood_other_last_date":"last_trans_dt",
                                    "hiv_transplant":"rcvd_transpl","hiv_healthcare_setting":"hlth_worker","other_document_risk":"other_risk"})
        
        #display(dfQA)
        
        dfQA = dfQA[['record_no','invunid','parent_id','rollup_no','doc_type','stateno','name_use_cd','f_name','m_name',
                     'l_name','addr_type_cd', 'addr_original_type_cd', 'address_dt', 'state', 'cntry', 'cntry_usd',
                     'birth_sex', 'birth_dt','birth_cntry', 'birth_usd', 'vital_status', 'swm',
                     'swf', 'idu', 'rcvd_clot_factor', 'factor_type', 'factor_dt', 'sexidu', 'sexbi', 'sex_transf',
                     'sexhemo','sex_transpl', 'sex_aidshiv', 'rcvd_transf',  'rcvd_transpl', 'hlth_worker', 'occupation',
                     'other_risk', 'nir', 'nir_dt', 'refer_obgyn']]
        
        #'first_trans_dt','last_trans_dt','sexhemo'
        
        #dfQA['parent_id'] = dfQA['parent_id'].astype(str)
        #dfQA['case_id'] = dfQA['case_id'].astype(str)
        #df1 = df[['CASE_ID','U_ID','parent_id','rollup_no','doc_type','STATENO','rcvd_dt','source','name_use_cd','f_name','m_name','l_name','birth_dt']]
        #df1['parent_id'] = df1['parent_id'].astype(str)
        #df1['CASE_ID'] = df1['CASE_ID'].astype(str)
        #df1.columns = df1.columns.str.lower()
    

                    
        #display(dfQA)
        
        dfQA.set_index('record_no', inplace = True)
        
        #dfQA = pd.merge(df1, dfQA, how = 'inner', on=['case_id', 'parent_id']).reset_index(drop=True)
    
        #dfQA = pd.concat([df1, dfQA], keys=['case_id','parent_id'], axis=1)
        
        display(dfQA)

KeyError: 'CASE_ID'